In [1]:
print("ok")

ok


In [2]:
%pwd

'c:\\Users\\user\\end-to-end-medical-chatbot-generative-ai\\research'

In [3]:
import os
os.chdir("../")

In [4]:
from langchain.document_loaders import PyPDFLoader,DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter


In [5]:

#Extract Data From the PDF File
def load_pdf_file(Data):
    loader= DirectoryLoader(Data,
                            glob="*.pdf",
                            loader_cls=PyPDFLoader)

    documents=loader.load()

    return documents

In [6]:
extracted_data=load_pdf_file("Data/")

In [7]:
#extracted_data

In [8]:
#split data into chunks
def text_split(extracted_data):
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks=text_splitter.split_documents(extracted_data)
    return text_chunks

In [9]:


text_chunks=text_split(extracted_data)
print("Length of Text Chunks", len(text_chunks))

Length of Text Chunks 5860


In [10]:
from langchain.embeddings import HuggingFaceBgeEmbeddings


In [11]:
def download_hugginng_face_embeddings():
    embeddings=HuggingFaceBgeEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [12]:
from langchain_community.embeddings import HuggingFaceBgeEmbeddings

def download_hugging_face_embeddings():
    embeddings = HuggingFaceBgeEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

# Call the function
embeddings = download_hugging_face_embeddings()
print("Embeddings model loaded successfully!")


C:\Users\user\AppData\Local\Temp\ipykernel_16556\3415889756.py:4: LangChainDeprecationWarning: The class `HuggingFaceBgeEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceBgeEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
c:\Users\user\anaconda3\envs\medibot\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Embeddings model loaded successfully!


In [13]:
query_result=embeddings.embed_query("Hello World")
print("length",len(query_result))

length 384


In [14]:

from dotenv import load_dotenv
load_dotenv()

True

In [15]:
PINECONE_API_KEY=os.environ.get('PINECONE_API_KEY')
GROQ_API_KEY=os.environ.get('GROQ_API_KEY')

In [ ]:
from pinecone import Pinecone
from pinecone import ServerlessSpec
# Create an instance of the Pinecone class
pinecone = Pinecone(api_key="pcsk_5NDFLe_Cu9zcfCtfKuCPqqR3psruMY2WiAg9jahysSpe8ABmJ55zKR5761nPpzFtUV6Tq1")



# Create an index
index_name = "medibot"
if index_name not in pinecone.list_indexes():
    pinecone.create_index(index_name, dimension=384, metric="cosine", spec=ServerlessSpec(
        cloud="aws", 
        region="us-east-1"
    ) )
index = pinecone.Index(index_name)


In [28]:
index = pinecone.Index("medibot") 


In [29]:
import os 
os.environ["PINECONE_API_KEY"]="pcsk_5NDFLe_Cu9zcfCtfKuCPqqR3psruMY2WiAg9jahysSpe8ABmJ55zKR5761nPpzFtUV6Tq1"
os.environ['GROQ_API_KEY'] = "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"

In [30]:
from langchain_groq import ChatGroq
llm = ChatGroq(api_key=os.getenv("GROQ_API_KEY"))    

In [ ]:
# Embed each chunk and upset the embeddings into your pinecone index
from langchain_pinecone import PineconeVectorStore
docsearch=PineconeVectorStore.from_documents(
    documents=text_chunks,
    index_name=index_name,
    embedding=embeddings)


In [ ]:
#Load Existing Index
from langchain_pinecone import PineconeVectorStore
docsearch = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embeddings) 


In [ ]:
retriever=docsearch.as_retriever(search_type="similarity",search_kwargs={"k":3})


In [ ]:
retrieved_docs=retriever.invoke("what is Acne")

In [ ]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate


system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)


prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [ ]:

question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [ ]:
response = rag_chain.invoke({"input": "what is Acromegaly and gigantism?"})
print(response["answer"])

Acromegaly and gigantism are disorders caused by the abnormal release of growth hormone (GH) from the pituitary gland, leading to increased growth in bone and soft tissue. Diagnosis is based on laboratory tests measuring an increase in IGF-1 (3-10 times above the normal level), but these results must be carefully interpreted based on the patient's condition.
